In [ ]:
!pip install pyyaml
!pip install roboflow
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

# Train

In [ ]:
!ls -la {HOME}

In [ ]:
%cd {HOME}

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="cKqZpnnGNlnANsaMDaH6")
project = rf.workspace("first-workspace-tuukh").project("dental-psqg6")
version = project.version(3)
dataset = version.download("yolov7")


In [ ]:
!ls {HOME}

In [ ]:
%cd {HOME}

!python train.py \
--batch 16 --epochs 30 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

# Test kết quả

In [ ]:
!ls {HOME}/runs/train/exp3/

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/runs/train/exp4/results.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/runs/train/exp4/F1_curve.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/runs/train/exp3/P_curve.png", width=1000)


In [ ]:
Image(filename=f"{HOME}/runs/train/exp3/R_curve.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/runs/train/exp4/confusion_matrix.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/runs/train/exp3/val_batch0_pred.jpg", width=1000)

In [ ]:
%cd {HOME}

!python val.py \
--img 640 --batch 16 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/runs/train/exp3/weights/best.pt

In [ ]:
!python detect.py \
--img 640 --conf 0.1 --device 0 \
--weights {HOME}/runs/train/exp4/weights/best.pt \
--source {dataset.location}/test/images

In [ ]:
import glob

from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/exp3/*.jpg')[:]:
      display(Image(filename=image_path, width=600))

# Deploy với Inference

In [ ]:
!pip install -q inference supervision

In [ ]:
!pip install inference-sdk

In [ ]:
from inference_sdk import InferenceHTTPClient, InferenceConfiguration
import cv2
import random
#Cái này để chỉnh threshold output, do model hơi yếu nên chỉnh threshold thấp xíu (sẽ giảm accuracy)
custom_configuration = InferenceConfiguration(confidence_threshold=0.2)

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="cKqZpnnGNlnANsaMDaH6"
)
image_paths = sv.list_files_with_extensions(
    directory=f"{dataset.location}/valid/images",
    extensions=['png', 'jpg', 'jpeg']
)
image_path = random.choice(image_paths)
image = cv2.imread(image_path)

CLIENT.configure(custom_configuration)
result = CLIENT.infer(image, model_id="dental-psqg6/3")
result

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def plot_detections(image_path, detections):
    """
    Plots bounding boxes on an image based on detection results.

    Parameters:
    - image_path (str): The path to the image file.
    - detections (dict): The detection results containing bounding boxes and other details.

    Example of `detections` structure:
    {
        'time': 0.31112832799999524,
        'image': {'width': 640, 'height': 640},
        'predictions': [
            {'x': 242.0, 'y': 416.0, 'width': 38.0, 'height': 104.0, 'confidence': 0.4508063793182373, 'class': 'caries'},
            ...
        ]
    }
    """

    # Load the image
    image = Image.open(image_path)

    # Create a matplotlib figure and axis
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(image)

    # Iterate over the detections and draw bounding boxes
    for pred in detections['predictions']:
        # Extract bounding box details
        x = pred['x'] - pred['width'] / 2  # Convert center x to top-left x
        y = pred['y'] - pred['height'] / 2  # Convert center y to top-left y
        width = pred['width']
        height = pred['height']
        confidence = pred['confidence']
        class_name = pred['class']
        
        # Create a rectangle patch
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        
        # Add the rectangle patch to the plot
        ax.add_patch(rect)
        
        # Add text label
        plt.text(x, y, f'{class_name}: {confidence:.2f}', color='red', fontsize=12, weight='bold')

    # Set the title of the plot
    plt.title('Object Detection Results')

    # Show the plot with bounding boxes
    plt.show()

In [ ]:
plot_detections(image_path,result)